# Run analysis on multiple files

In [19]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing
import h5py

In [33]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FinerResolution_BendingStiffness'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-23'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-18'
# data_folder = '/home/deepak/Dropbox/LacryModeling/ModellingResults/StochasticActivity/NormalDistribution'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_750'

data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FilamentLength_Sweeps/activity_time_750'

print(os.listdir(data_folder))

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.' and 'analysis' not in fileNames):
            files_list.append(os.path.join(dirs,fileNames))
#         if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
#             files_list.append(os.path.join(dirs,fileNames))

print('No:of Simulation files: ', len(files_list))

def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)


    root_folder, file_name = os.path.split(file)

    # Create a data-packet to save for further analysis
    # Collect simulation metadata
#     metadata_path = os.path.join(root_folder, 'metadata.csv')
#     assert(os.path.exists(metadata_path))
#     df = pd.read_csv(metadata_path)
    
    # @@@ Better to use Metadata from the HDF5 file so there is no possible delinking of data and metadata
    df = pd.DataFrame({'N particles':[filament.Np],'radius':[filament.radius],'bond length':[filament.b0],
                       'spring constant':[filament.k], 'kappa_hat':[filament.kappa_hat], 
                       'force strength':[filament.F0], 'stresslet strength':[filament.S0], 
                       'potDipole strength':[filament.D0], 'simulation type':[filament.sim_type], 
                       'boundary condition 0':[filament.bc[0]], 'boundary condition 1':[filament.bc[-1]], 
                       ' activity time scale':[filament.activity_timescale], 'viscosity':[filament.mu]})
    
    
    periodic_flag = None 
    min_period = None
    
    filament.filament_tip_coverage(save = True)
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) == int(df[' activity time scale']*500)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period = filament.classify_filament_dynamics()
    else:
        simulation_completed = False

    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    df['period'] = min_period
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    
    
    
    
    # Calculate the filament tip angles (at constant phase) and save them to a file
    
    phase = 0
    angles = filament.compute_tip_angle_at_constant_phase(phase_value = phase)
    print(angles)
    
    

    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
    df.to_csv(os.path.join(save_folder, save_file))
    
    # Create a dataset and save the data
    save_file = file_name[:-5] + '_analysis.hdf5'


    with h5py.File(os.path.join(save_folder, save_file), "w") as f:
        dset = f.create_group("analysis data")
        dset.attrs['N particles'] = filament.Np
        dset.attrs['radius'] = filament.radius
        dset.attrs['bond length'] = filament.b0
        dset.attrs['spring constant'] = filament.k
        dset.attrs['kappa_hat'] = filament.kappa_hat
        dset.attrs['force strength'] = filament.F0
        dset.attrs['stresslet strength'] = filament.S0
        dset.attrs['potDipole strength'] = filament.D0
        dset.attrs['simulation type'] = filament.sim_type
        dset.attrs['activity time scale'] = filament.activity_timescale
        dset.attrs['viscosity'] = filament.mu
        dset.attrs['boundary condition 0'] = filament.bc[0]
        dset.attrs['boundary condition 1'] = filament.bc[-1]
        dset.attrs['Sim complete'] = simulation_completed
        
        dset.attrs['constant phase'] = phase
        dset.create_dataset("Tip angles", data = angles)
    
    

['SimResults_Np_16_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_128_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_64_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_256_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_96_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point', 'SimResults_Np_48_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point']
No:of Simulation files:  21


In [34]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))


100%|██████████| 21/21 [00:00<00:00, 208.98it/s]
[Parallel(n_jobs=12)]: Done   4 out of  21 | elapsed:   10.6s remaining:   45.0s
[Parallel(n_jobs=12)]: Done   7 out of  21 | elapsed:   19.4s remaining:   38.9s
[Parallel(n_jobs=12)]: Done  10 out of  21 | elapsed:   22.3s remaining:   24.6s
[Parallel(n_jobs=12)]: Done  13 out of  21 | elapsed:   24.7s remaining:   15.2s
[Parallel(n_jobs=12)]: Done  16 out of  21 | elapsed:   27.9s remaining:    8.7s
[Parallel(n_jobs=12)]: Done  19 out of  21 | elapsed:   31.2s remaining:    3.3s
[Parallel(n_jobs=12)]: Done  21 out of  21 | elapsed:   31.9s finished
